<a href="https://colab.research.google.com/github/samarrkilfaden-droid/advanced_ai_exercises/blob/main/Day4_task1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import torch, torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [17]:
# ---------------------------
# Data
# ---------------------------
transform = transforms.Compose([
    transforms.Resize(32),
    transforms.ToTensor()              # BUG
])
ds = torchvision.datasets.CIFAR10('./data', download=True, train=True, transform=transform)
loader = DataLoader(ds, batch_size=128, shuffle=True, num_workers=2, pin_memory=True)

In [18]:
# ---------------------------
# Hyperparams
# ---------------------------
z_dim = 100
g_lr  = 2e-4  # BUG
d_lr  = 2e-4


In [25]:

# ---------------------------
# Models
# ---------------------------
class D(nn.Module):
    def __init__(self, ch=64):
        super().__init__()
        self.z_dim = z_dim
        self.net = nn.Sequential(
            nn.Conv2d(3, ch, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),  # Changed to LeakyReLU
            nn.Conv2d(ch, ch*2, 4, 2, 1),
            nn.BatchNorm2d(ch*2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ch*2, ch*4, 4, 2, 1),
            nn.BatchNorm2d(ch*4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ch*4, 1, 4, 1, 0)
            # Removed Sigmoid
        )

    def forward(self, x):
        return self.net(x).view(x.size(0))

class G(nn.Module):
    def __init__(self, z=100, ch=64):
        super().__init__()
        self.z_dim = z_dim
        self.net = nn.Sequential(
            nn.ConvTranspose2d(z,   ch*4, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ch*4), nn.ReLU(True),
            nn.ConvTranspose2d(ch*4, ch*2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ch*2), nn.ReLU(True),
            nn.ConvTranspose2d(ch*2, ch,   4, 2, 1, bias=False),
            nn.BatchNorm2d(ch),   nn.ReLU(True),
            nn.ConvTranspose2d(ch,  3,     4, 2, 1, bias=False),
            nn.Tanh()
            # BUG
        )
    def forward(self, z):
        z = z.view(z.size(0), self.z_dim , 1, 1)   # BUG
        return self.net(z)

Dnet = D().to(device)
Gnet = G(z_dim).to(device)


In [26]:

# ---------------------------
# Loss & Optimizers
# ---------------------------
crit = nn.BCEWithLogitsLoss()             # using logits loss → D must NOT have Sigmoid
opt_d = torch.optim.Adam(Dnet.parameters(), lr=d_lr, betas=(0.9, 0.999))
opt_g = torch.optim.Adam(Gnet.parameters(), lr=g_lr,  betas=(0.9, 0.999))


In [29]:

# ---------------------------
# Training (intentionally wrong)
# ---------------------------

Gnet.train()
Dnet.train()

for real, _ in loader:
    real = real.to(device)
    b = real.size(0)

    opt_d.zero_grad(set_to_none=True)


    # ---- Discriminator step (generating a batch of fake images) ----
    z = torch.randn(b, z_dim, device=device)#This creates a tensor of shape (b, z_dim) filled with random values sampled from a standard normal distribution/# Sample random latent vectors
    fake = Gnet(z.view(b,z_dim,1,1)) # Generate fake images from the latent vectors

    # BUG
    loss_d = crit(Dnet(real), torch.ones(b, device=device)) +\
    crit(Dnet(fake), torch.zeros(b, device=device))

    loss_d.backward()
    opt_d.step()                           # BUG

    opt_g.zero_grad(set_to_none=True)
    # ---- Generator step ----
    z = torch.randn(b, z_dim, device=device)
    fake = Gnet(z.view(b,z_dim,1,1))
    # BUG
    loss_g = crit(Dnet(fake), torch.ones(b, device=device))
    loss_g.backward()
    opt_g.step()                           # BUG

    break  # keep single mini-batch for this broken demo

print("Your task: fix all 10 bugs until training runs and samples look reasonable.")

Your task: fix all 10 bugs until training runs and samples look reasonable.
